# This notebook performs topic modeling on lyrics to that we can investigate questions including : The differences between East Coast and West Coast rap

In [1]:
import sys
import os

In [2]:
import gensim
import nltk
import pandas as pd
import seaborn as sns

c:\anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
# NOTE : Without setting the engine here, we might hit the exception : "C error: EOF inside string ..."
lyrics_df = pd.read_csv('c:/datasets/lyrics/lyrics.csv',
                       engine = 'python')

In [26]:
#replace carriage returns
lyrics_df = lyrics_df.replace({'\n': ' '}, regex=True)

In [27]:
print('Total size of lyrics dataset : {}'.format(len(lyrics_df)))

Total size of lyrics dataset : 362237


In [28]:
lyrics_df.head(10)

,index,song,year,artist,genre,lyrics
0,0,ego-remix,2009,beyonce-knowles,Pop,"Oh baby, how you doing? You know I'm gonna cut..."
1,1,then-tell-me,2009,beyonce-knowles,Pop,"playin' everything so easy, it's like you seem..."
2,2,honesty,2009,beyonce-knowles,Pop,If you search For tenderness It isn't hard to ...
3,3,you-are-my-rock,2009,beyonce-knowles,Pop,"Oh oh oh I, oh oh oh I [Verse 1:] If I wrote a..."
4,4,black-culture,2009,beyonce-knowles,Pop,"Party the people, the people the party it's po..."
5,5,all-i-could-do-was-cry,2009,beyonce-knowles,Pop,I heard Church bells ringing I heard A choir s...
6,6,once-in-a-lifetime,2009,beyonce-knowles,Pop,This is just another day that I would spend Wa...
7,7,waiting,2009,beyonce-knowles,Pop,"Waiting, waiting, waiting, waiting Waiting, wa..."
8,8,slow-love,2009,beyonce-knowles,Pop,[Verse 1:] I read all of the magazines while w...
9,9,why-don-t-you-love-me,2009,beyonce-knowles,Pop,"N-n-now, honey You better sit down and look ar..."


In [29]:
lyric_text_list = lyrics_df['lyrics'].tolist()
lyrics_index_list = lyrics_df.index.values

In [32]:
lyrics_df.ix[158]

c:\anaconda3\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  if __name__ == '__main__':


index                 158
song             lemonade
year                 2016
artist    beyonce-knowles
genre                 Pop
lyrics                NaN
Name: 158, dtype: object

In [34]:
%%time

lyrics_tokenized_list = []
token_count_list = []
unique_token_count_list = []
for i, text in enumerate(lyric_text_list):
    if i % 10000 == 0:
        print('Tokenization process : [{0}/{1}]'.format(i, len(lyric_text_list)))
        
    # get the index into the original text
    index = lyrics_index_list[i]
        
    # there are lots and lots of rows which have no lyrics at all, so let's skip them
    if not isinstance(text, str):
        #print('Skipping column type : {0} at index {1}'.format(type(text), index))  
        continue
        
    # this is a better way to tokenize, but for the interest of time, we will tokenize with
    # whitespace using python's split() function
    #tokens = nltk.word_tokenize(text)
    tokens = text.split()
    lyrics_tokenized_list.append(tokens)
    token_count_list.append(len(tokens))
    unique_token_count = set(tokens)
    unique_token_count_list.append(unique_token_count)
    
print('DONE reading, tokenizing and counting')

Tokenization process : [0/362237]
Tokenization process : [10000/362237]
Tokenization process : [20000/362237]
Tokenization process : [30000/362237]
Tokenization process : [40000/362237]
Tokenization process : [50000/362237]
Tokenization process : [60000/362237]
Tokenization process : [70000/362237]
Tokenization process : [80000/362237]
Tokenization process : [90000/362237]
Tokenization process : [100000/362237]
Tokenization process : [110000/362237]
Tokenization process : [120000/362237]
Tokenization process : [130000/362237]
Tokenization process : [140000/362237]
Tokenization process : [150000/362237]
Tokenization process : [160000/362237]
Tokenization process : [170000/362237]
Tokenization process : [180000/362237]
Tokenization process : [190000/362237]
Tokenization process : [200000/362237]
Tokenization process : [210000/362237]
Tokenization process : [220000/362237]
Tokenization process : [230000/362237]
Tokenization process : [240000/362237]
Tokenization process : [250000/362237]
